In [ ]:
import os
import subprocess


# Define the names of the GULP input files to run
path = "/project/wen/emmanuel/gulp/Aluminosilicate_redo"
os.chdir(path)

# Loop over the input files and create a separate submission script for each one
for input_file in os.listdir():
    if not input_file.endswith('.gin'): 
        continue
    # Create the directory to save the files
    output_dir = path + '/' + input_file[:-3]
    

    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    os.rename(f"{path}/{input_file}", f"{output_dir}/{input_file}")

    # Create the contents of the submission script
    submit_script = f"""#!/bin/bash
#SBATCH --job-name={input_file}
#SBATCH --output={input_file[:-3]}.out
#SBATCH --error={input_file[:-3]}.err
#SBATCH --time=48:00:00
#SBATCH -N 1
#SBATCH -n 12
#SBATCH --mem-per-cpu=3G

# Run the GULP input file
module load intel-oneapi
module load gulp/6.1.2
mpirun gulp < {input_file} 
"""

    # Write the submission script to a file
    with open(f"{output_dir}/submit_{input_file[:-4]}.sh", 'w') as f:
        f.write(submit_script)

    # Submit the job using the subprocess module
    subprocess.run(['sbatch', f"submit_{input_file[:-4]}.sh"], cwd=output_dir)


